In [244]:
#Import libraries
import os
import numpy as np
import random
from numpy import asarray
from keras import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from PIL import ImageFilter
from sklearn.model_selection import train_test_split

In [245]:
#Paths for images
train_not_hot_dog = "train\\not_hot_dog"
train_hot_dog = "train\\hot_dog"
test_not_hot_dog = "test\\not_hot_dog"
test_hot_dog = "test\\hot_dog"

In [246]:
img_size = 64
channels = 1

In [247]:
#Function for dowloading and prepping images
def load_img_data(directory):
    
    counter = 0
    images = []
    
    filenames = os.listdir(directory)
    
    
    for filename in filenames:
        img = Image.open(directory + "\\" + filename)
        img = img.convert('L')
        img = img.resize((img_size, img_size), Image.ANTIALIAS)
        img = img.filter(ImageFilter.SHARPEN)
        img = img.filter(ImageFilter.GaussianBlur(radius=2))
        #img.show()
        img = (asarray(img))
        img = img.astype(float)
        img = img.reshape(img_size, img_size, channels)        
        images.append(img)
    
    return images

In [248]:
#Dowload the images with the function declared above

train_img_not_hot_dog = load_img_data(train_not_hot_dog)
print("1 complete")
train_img_hot_dog = load_img_data(train_hot_dog)
print("2 complete")
test_img_not_hot_dog = load_img_data(test_not_hot_dog)
print("3 complete")
test_img_hot_dog = load_img_data(test_hot_dog)
print("4 complete")



1 complete
2 complete
3 complete
4 complete


In [249]:
#Setup labels for training and testing
training_size = len(train_img_not_hot_dog) + len(train_img_hot_dog)
test_size = len(test_img_not_hot_dog) + len(test_img_hot_dog)

training_labels_not_hot_dog = np.full((len(train_img_not_hot_dog), 2), (0, 1))
training_labels_hot_dog = np.full((len(train_img_hot_dog), 2), (1, 0))
training_labels = np.concatenate([training_labels_not_hot_dog, training_labels_hot_dog])

test_labels_not_hot_dog = np.full((len(test_img_not_hot_dog), 2), (0, 1))
test_labels_hot_dog = np.full((len(test_img_hot_dog), 2), (1, 0))
test_labels = np.concatenate([test_labels_not_hot_dog, test_labels_hot_dog])


In [250]:
#Setup training images and test images
training_images = np.concatenate([train_img_not_hot_dog, train_img_hot_dog])
test_images = np.concatenate([test_img_not_hot_dog, test_img_hot_dog])

In [251]:
#Make RGB-channels to one vector
training_images[277].shape


(64, 64, 1)

In [252]:
#Augment data
datagen = ImageDataGenerator(
                            rescale=1.0/255.0,
                            rotation_range=15,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            shear_range=0.01,
                            zoom_range=[0.3, 1.25],
                            horizontal_flip=True,
                            brightness_range=[0.5, 1.5],
                            )
train_iterator = datagen.flow(
                              training_images, 
                              training_labels, 
                              batch_size=50)



In [253]:
#Define model function
def define_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same', input_shape=(img_size, img_size, channels)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(2, activation='softmax'))
    opt = Adam(lr=0.001)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [254]:
test_imagesf, valid_imagesf, test_labelsf, valid_labelsf = train_test_split(test_images, test_labels, test_size=0.4, shuffle= True)

In [255]:
model = define_model()
hisory = model.fit_generator(train_iterator, steps_per_epoch=(len(training_images // 500)), epochs=20, validation_data=(valid_imagesf, valid_labelsf))


#model = create_model()
#model.fit(training_imagesf, training_labelsf, epochs=5, verbose=1)

Epoch 1/20
446/498 [=========================>....] - ETA: 26s - loss: 0.6988 - acc: 0.5168

KeyboardInterrupt: 

In [230]:
#Testing parameters
ti = test_images
tl = test_labels

In [231]:
#Test model
datagen_test = ImageDataGenerator(rescale=1.0/255.0)
test_generator = datagen_test.flow(ti, batch_size=1, shuffle=False)
test_generator.reset()
results = model.predict_generator(test_generator, steps=len(test_labels))

In [232]:
conv_test_labels = convert_output_labels(tl)
conv_pred_labels = convert_output_labels(results)

In [233]:
conv_pred_labels

array([[0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],

In [234]:
#conv_test_labels

In [235]:
#Final test score
evaluate(conv_test_labels, conv_pred_labels)

0.656

In [189]:
#Functions for evaluation
def evaluate(predicted_labels, ground_labels):
    
    set_length = len(ground_labels)
    correct_labels = 0
    
    for i in range(0, set_length):
        if predicted_labels[i] == ground_labels[i]:
            correct_labels += 1
    return (correct_labels/set_length)

def convert_output_labels(data):
    
    labels = np.zeros(shape=[len(data), 1])
    
    for i in range(0, len(data)):
        labels[i]= np.argmax(data[i])
    return labels